### Imports

In [155]:
import pandas as pd
import os
import pprint

### View data

In [156]:
file_path = "../raw data"
csv_files = os.listdir(file_path)

In [157]:
%%time
df_list = []
standard_format_files = set(['whistler.csv','squaw.csv','snowbird.csv','mtBaker.csv','jackson.csv','bigsky.csv'])
for csv_file in csv_files:
    if csv_file in standard_format_files:
        df_ = pd.read_csv(file_path+"/"+csv_file,parse_dates={'date':[0]}, infer_datetime_format=True)
        df_list.append(df_)
snow_data = pd.concat(df_list)
#snow_data.fillna(0,inplace=True)
display(snow_data.head())

,date,mountain,max
0,1992-01-01,Big Sky,16.5
1,1993-01-01,Big Sky,21.4
2,1994-01-01,Big Sky,13.8
3,1995-01-01,Big Sky,28.6
4,1996-01-01,Big Sky,30.7


CPU times: user 609 ms, sys: 5.84 ms, total: 615 ms
Wall time: 626 ms


### Check for Null Values

In [158]:
nulls = snow_data[snow_data.isnull().any(axis=1)]
print("Total rows w/ null values: {:,}".format(nulls.shape[0]))
nulls.head()

Total rows w/ null values: 1,415


,date,mountain,max
373,2001-09-07 16:00:00,whistler,NaN
374,2001-09-08 16:00:00,whistler,NaN
375,2001-09-09 16:00:00,whistler,NaN
376,2001-09-10 16:00:00,whistler,NaN
377,2001-09-11 16:00:00,whistler,NaN


### Data cleanse 

In [159]:
"date" in snow_data.columns

True

In [160]:
%%time
###
### unify timestamp
###
snow_data.loc[:,"day"] = pd.to_datetime(snow_data.date.dt.strftime('%Y-%m-%d'), format='%Y-%m-%d')
###
### review
###
print("REVIEW:")
display(snow_data.loc[snow_data.loc[:,"mountain"]=="whistler",].head())

REVIEW:


,date,mountain,max,day
0,2000-08-30 16:00:00,whistler,0.0,2000-08-30
1,2000-08-31 16:00:00,whistler,0.0,2000-08-31
2,2000-09-01 16:00:00,whistler,0.0,2000-09-01
3,2000-09-02 16:00:00,whistler,0.0,2000-09-02
4,2000-09-03 16:00:00,whistler,0.0,2000-09-03


CPU times: user 51.6 ms, sys: 2.84 ms, total: 54.4 ms
Wall time: 52.2 ms


### group by day

In [161]:
###
### group by day
###
print("GROUP BY DAY:")
snow_data_g = snow_data.groupby(by=["day"]).agg({"mountain":["unique","nunique"],"max":"unique"})
display(snow_data_g.head())

GROUP BY DAY:


mountain              max
             unique nunique   unique
day                                 
1981-01-01  [squaw]       1   [38.4]
1982-01-01  [squaw]       1   [92.1]
1983-01-01  [squaw]       1  [110.2]
1984-01-01  [squaw]       1   [49.6]
1985-01-01  [squaw]       1   [33.1]

### create data container

In [162]:
%%time
###
### create data container
###
unique_mtn_values = snow_data.mountain.unique().tolist()
data = {mtn:[] for i,mtn in enumerate(unique_mtn_values)}
data["date"] = []
display(data)


{'Big Sky': [],
 'squaw': [],
 'whistler': [],
 'Mt. Baker': [],
 'Jackson Hole': [],
 'snowbird': [],
 'date': []}

CPU times: user 2.32 ms, sys: 965 µs, total: 3.29 ms
Wall time: 2.34 ms


### finalize data format for final df

In [163]:
###
### finalize data format for final df
### 
def fill_in_missing_snow_values(row, data,unique_mtn_values):
    d = dict(zip(row.mountain["unique"],row["max"]["unique"]))
    for mtn in unique_mtn_values:
        if mtn in d:
            data[mtn].append(d[mtn])
        else:
            data[mtn].append(None)
    data["date"].append(row.name)
    
snow_data_g.apply(fill_in_missing_snow_values,axis=1,args=[data,unique_mtn_values])
final_struct = pd.DataFrame(data)
final_struct.set_index("date",inplace=True)
print("FINAL DATAFRAME STRUCTURE:")
display(final_struct.head())

FINAL DATAFRAME STRUCTURE:


,Big Sky,squaw,whistler,Mt. Baker,Jackson Hole,snowbird
date,,,,,,
1981-01-01,NaN,38.4,NaN,NaN,NaN,NaN
1982-01-01,NaN,92.1,NaN,NaN,NaN,NaN
1983-01-01,NaN,110.2,NaN,NaN,NaN,NaN
1984-01-01,NaN,49.6,NaN,NaN,NaN,NaN
1985-01-01,NaN,33.1,NaN,NaN,NaN,NaN


### fill in missing dates

In [164]:
###
### fill in missing dates
### 
def custom_resampler(array):
    try: 
        value = sum(array)
    except TypeError:
        value = None
    return value
final_df = final_struct.resample('D').apply(custom_resampler)
print("FiNAL DATAFRAME:")
display(final_df)

FiNAL DATAFRAME:


,Big Sky,squaw,whistler,Mt. Baker,Jackson Hole,snowbird
date,,,,,,
1981-01-01,NaN,38.4,NaN,NaN,NaN,NaN
1981-01-02,0.0,0.0,0.000000,0.0,0.0,0.0
1981-01-03,0.0,0.0,0.000000,0.0,0.0,0.0
1981-01-04,0.0,0.0,0.000000,0.0,0.0,0.0
1981-01-05,0.0,0.0,0.000000,0.0,0.0,0.0
1981-01-06,0.0,0.0,0.000000,0.0,0.0,0.0
1981-01-07,0.0,0.0,0.000000,0.0,0.0,0.0
1981-01-08,0.0,0.0,0.000000,0.0,0.0,0.0
1981-01-09,0.0,0.0,0.000000,0.0,0.0,0.0


### Munge & save D3js data

In [169]:
d3_df = final_df.reset_index()
d3_data_format = d3_df.to_json(orient='records')
d3_df.to_json("../d3js_data/d3js_data.json",orient='records')
pprint.pprint(d3_data_format)

('[{"date":347155200000,"Big Sky":null,"squaw":38.4,"whistler":null,"Mt. '
 'Baker":null,"Jackson Hole":null,"snowbird":null},{"date":347241600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":347328000000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":347414400000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":347500800000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":347587200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":347673600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":347760000000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":347846400000,"Big '

 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":443491200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":443577600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":443664000000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":443750400000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":443836800000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":443923200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":444009600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":444096000000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler

 'Hole":0.0,"snowbird":0.0},{"date":529804800000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":529891200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":529977600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":530064000000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":530150400000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":530236800000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":530323200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":530409600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"da

 'Hole":0.0,"snowbird":0.0},{"date":627523200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":627609600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":627696000000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":627782400000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":627868800000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":627955200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":628041600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":628128000000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"da

 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":724204800000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":724291200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":724377600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":724464000000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":724550400000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":724636800000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":724723200000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler":0.0,"Mt. Baker":0.0,"Jackson '
 'Hole":0.0,"snowbird":0.0},{"date":724809600000,"Big '
 'Sky":0.0,"squaw":0.0,"whistler

...

 'Hole":null,"snowbird":null},{"date":1131494400000,"Big '
 'Sky":null,"squaw":null,"whistler":5.472440945,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1131580800000,"Big '
 'Sky":null,"squaw":null,"whistler":6.220472441,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1131667200000,"Big '
 'Sky":null,"squaw":null,"whistler":6.811023622,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1131753600000,"Big '
 'Sky":null,"squaw":null,"whistler":7.129658793,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1131840000000,"Big '
 'Sky":null,"squaw":null,"whistler":7.401574803,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1131926400000,"Big '
 'Sky":null,"squaw":null,"whistler":7.401574803,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1132012800000,"Big '
 'Sky":null,"squaw":null,"whistler":7.401574803,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1132099200000,"Bi

 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1220918400000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1221004800000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1221091200000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1221177600000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1221264000000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1221350400000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1221436800000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"dat

 'Hole":null,"snowbird":null},{"date":1328832000000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1328918400000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1329004800000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1329091200000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1329177600000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1329264000000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1329350400000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1329436800000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt.

 'Hole":null,"snowbird":null},{"date":1436832000000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1436918400000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1437004800000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1437091200000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1437177600000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1437264000000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1437350400000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1437436800000,"Big '
 'Sky":null,"squaw":null,"whistler":null,"Mt.

 'Sky":null,"squaw":null,"whistler":12.55905512,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1544832000000,"Big '
 'Sky":null,"squaw":null,"whistler":12.75590551,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1544918400000,"Big '
 'Sky":null,"squaw":null,"whistler":13.34645669,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1545004800000,"Big '
 'Sky":null,"squaw":null,"whistler":15.15748031,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1545091200000,"Big '
 'Sky":null,"squaw":null,"whistler":16.2992126,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1545177600000,"Big '
 'Sky":null,"squaw":null,"whistler":16.88976378,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1545264000000,"Big '
 'Sky":null,"squaw":null,"whistler":18.18897638,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":1545350400000,"Big '
 'Sky":null,"squaw":null,"whistler":19.60629921,"Mt. Bak